In [4]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
#import logging
#logging.basicConfig(level=logging.INFO)

In [5]:
# Load pre-trained model (weights) and tokenizer
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
def fill_in_the_blank(text, masked_index):
    # fills in a blank in a sentence
    # text, the full unmasked text on which to make the prediction
    # masked_index, the word/token index that you want the model to guess
    text = '[CLS] '+text+' [SEP]'
    tokenized_text = tokenizer.tokenize(text)
    print(' '.join(tokenized_text))
    tokenized_text[masked_index] = '[MASK]'
    print(' '.join(tokenized_text))
    segments_ids = [0 for i in tokenized_text]
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)

    predicted_index = torch.argmax(predictions[0, masked_index]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    tokenized_text[masked_index] = predicted_token
    print(' '.join(tokenized_text))

In [7]:
# fill in the blank examples
# original
# masked
# predicted
text = "sally sells seas shells down by the sea shore."
tokenizer.tokenize(text)
for masked_index in range(len(tokenizer.tokenize(text))+2):
    fill_in_the_blank(text, masked_index)
    print()

[CLS] sally sells seas shells down by the sea shore . [SEP]
[MASK] sally sells seas shells down by the sea shore . [SEP]
. sally sells seas shells down by the sea shore . [SEP]

[CLS] sally sells seas shells down by the sea shore . [SEP]
[CLS] [MASK] sells seas shells down by the sea shore . [SEP]
[CLS] he sells seas shells down by the sea shore . [SEP]

[CLS] sally sells seas shells down by the sea shore . [SEP]
[CLS] sally [MASK] seas shells down by the sea shore . [SEP]
[CLS] sally ##ing seas shells down by the sea shore . [SEP]

[CLS] sally sells seas shells down by the sea shore . [SEP]
[CLS] sally sells [MASK] shells down by the sea shore . [SEP]
[CLS] sally sells the shells down by the sea shore . [SEP]

[CLS] sally sells seas shells down by the sea shore . [SEP]
[CLS] sally sells seas [MASK] down by the sea shore . [SEP]
[CLS] sally sells seas ##hell down by the sea shore . [SEP]

[CLS] sally sells seas shells down by the sea shore . [SEP]
[CLS] sally sells seas shells [MASK] b